## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-26-21-00-55 +0000,nypost,"Stunned upstate NY mom gives birth to massive,...",https://nypost.com/2026/02/26/us-news/stunned-...
1,2026-02-26-21-00-00 +0000,wsj,Columbia University Student Detained by ICE,https://www.wsj.com/us-news/education/columbia...
2,2026-02-26-20-59-00 +0000,missionlocal,Mayor Lurie turned the anti-Jewish rantings of...,https://missionlocal.org/2026/02/sf-daniel-lur...
3,2026-02-26-20-57-59 +0000,nyt,Mamdani and Trump Discuss Housing at ‘Producti...,https://www.nytimes.com/2026/02/26/nyregion/ma...
4,2026-02-26-20-56-00 +0000,wsj,Treasury Yields Decline Ahead of U.S. Producer...,https://www.wsj.com/finance/investing/jgbs-fal...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2591/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
31,trump,54
159,epstein,18
152,iran,14
550,million,10
388,new,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
140,2026-02-26-16-49-03 +0000,nypost,Hillary Clinton accuses GOP of covering up for...,https://nypost.com/2026/02/26/us-news/hillary-...,112
54,2026-02-26-19-24-19 +0000,nypost,DOJ probes whether any Epstein files about unp...,https://nypost.com/2026/02/26/us-news/doj-prob...,107
6,2026-02-26-20-53-08 +0000,nypost,Columbia student detained by ICE will be relea...,https://nypost.com/2026/02/26/us-news/columbia...,102
206,2026-02-26-13-00-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...,101
284,2026-02-26-03-02-00 +0000,wsj,The Trump administration is pausing nearly $26...,https://www.wsj.com/politics/policy/vance-anno...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
140,112,2026-02-26-16-49-03 +0000,nypost,Hillary Clinton accuses GOP of covering up for...,https://nypost.com/2026/02/26/us-news/hillary-...
232,63,2026-02-26-11-22-15 +0000,cbc,Iran nuclear talks conclude with no deal annou...,https://www.cbc.ca/news/world/geneva-us-iran-n...
108,57,2026-02-26-17-48-00 +0000,wsj,Mortgage rates fell below 6% this week for the...,https://www.wsj.com/economy/housing/mortgage-r...
187,53,2026-02-26-14-25-08 +0000,bbc,A New York City snowball fight gone wrong leav...,https://www.bbc.com/news/articles/cwyxlgkj0k3o...
284,45,2026-02-26-03-02-00 +0000,wsj,The Trump administration is pausing nearly $26...,https://www.wsj.com/politics/policy/vance-anno...
143,45,2026-02-26-16-40-44 +0000,nypost,Warner Bros. Discovery revenue drops 6% as riv...,https://nypost.com/2026/02/26/media/warner-bro...
310,40,2026-02-26-00-26-00 +0000,wsj,Here’s a look at whether the math works on som...,https://www.wsj.com/politics/policy/the-math-b...
245,40,2026-02-26-10-39-25 +0000,nypost,Kim Jong Un calls South Korea ‘most hostile en...,https://nypost.com/2026/02/26/world-news/kim-j...
324,39,2026-02-25-23-48-45 +0000,nypost,Chip titan Nvidia posts record $68B in sales i...,https://nypost.com/2026/02/25/business/chip-ti...
355,38,2026-02-25-22-28-00 +0000,wsj,Spy agency blocks Congress from seeing Gabbard...,https://www.wsj.com/politics/policy/spy-agency...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
